# Zonas inundables

## 2_ Predicción

Este notebook genera la prediccion del modelo sobre un nuevo dataset. Para ello primero descargamos las imagenes satelitales y luego con satproc generamos el dataset de predicción con el formato necesario para el modelo de ML


**Definicion de los parametros para la predicción**

In [1]:
from unetseg.predict import PredictConfig, predict
from unetseg.evaluate import plot_data_results
import subprocess
import os

import glob

2022-07-05 17:38:26.246319: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [8]:

BASE_PATH = "../"
PATH_S1_IMAGES_PREDICT = f"{BASE_PATH}/images/"
#model
SIZE      = 160
STEP_SIZE = 160

MODEL_VERSION = "v0"
MODEL = f'UNet_TEST_160x160_spe100_7N_flood_aeras_colombia_{MODEL_VERSION}.h5'

BUCKET_RESULTS = "gs://dym-indunor-temp/immap/v3_1/floods_annotations/results/pred_500/"

### Descarga de las imagenes de predicción

Descarga las imagenes del bucket

### Satproc

Con esta herramienta genereamos un dataset de imágenes de predicción que, a diferencia del de entrenamiento, solo contiene imágenes.

In [9]:
path_to_files   = f'{PATH_S1_IMAGES_PREDICT}/*prediction*.tif' #carpeta a las imagenes
dataset_folder  = f'{BASE_PATH}/dataset/data_predict/pred_500/{str(SIZE)}_{str(STEP_SIZE)}/' #carpeta de destino del dataset


In [10]:

!satproc_extract_chips \
                $path_to_files \
                -o  $dataset_folder \
                --size $SIZE \
                --step-size $STEP_SIZE \
                --rescale \
                --rescale-mode percentiles  --lower-cut 0 --upper-cut 100

Rasters:   0%|                                            | 0/1 [00:00<?, ?it/s]
dataset_prediction_500m_epsg4326.tif windows: 100%|#| 216/216 [00:00<00:00, 1177
Rasters: 100%|###################################| 1/1 [00:00<00:00, 122.88it/s]


Pasamos la ruta de las imágenes de predicción y la ruta donde queremos que se guarde el resultado.



## Prediccion

Definimos la configuración para la predicción. Debemos pasar la ruta de las imágenes en el dataset de predicción, que es la ruta de salida de la línea anterior, y la correspondiente al modelo que entrenamos.

In [7]:
!ls ../data/weights/

UNet_TEST_160x160_spe100_7N_flood_aeras_colombia_v0.h5


In [11]:


predict_config = PredictConfig(

        images_path   = dataset_folder, # ruta a las imagenes sobre las cuales queremos predecir
        results_path  = f'{BASE_PATH}/dataset/data_results/pred_500/{MODEL_VERSION}/{str(SIZE)}_{str(STEP_SIZE)}/'
, # ruta de destino para nuestra predicción
        batch_size  = 4,
        model_path  = os.path.join(BASE_PATH +'data/weights', MODEL),  #  ruta al modelo (.h5)
        height = 160,
        width  = 160,
        n_channels = 6,
        n_classes  = 1,
        class_weights = [1])

#A continuación ejecutamos la predicción
predict(predict_config)    
#plot_data_results(num_samples=2, fig_size=(2, 2), predict_config=predict_config, img_ch =2, n_bands=3)

#create VTR
!rm tmp_list.txt

vrt_path        = f'{predict_config.results_path}/../all.vrt' # ruta de destino para nuestra predicción
tif_path        = f'{predict_config.results_path}/../all_{MODEL_VERSION}.tif'
predict_results = f'{predict_config.results_path}/*tif' # ruta de destino para nuestra predicción

filenames = glob.glob(predict_results)

with open('tmp_list.txt', 'w') as f:
    for line in filenames:
        f.write(line)
        f.write('\n')
!gdalbuildvrt -input_file_list tmp_list.txt $vrt_path

!gdal_translate -of GTiff -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 $vrt_path $tif_path


Total images to predict (..//dataset/data_predict/pred_500/160_160/images/*.tif): 216
After skipping existing results: 216


  0%|                                                                                                                                                                                | 0/54 [00:00<?, ?it/s]2022-07-05 17:40:08.535724: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-07-05 17:40:08.536762: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2299995000 Hz
2022-07-05 17:40:08.911359: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-07-05 17:40:10.180795: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-07-05 17:40:10.241018: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only 

Done!


0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 1915, 2785
0...10...20...30...40...50...60...70...80...90...100 - done.


In [12]:
!gsutil -m cp -r $tif_path $BUCKET_RESULTS 

Copying file://..//dataset/data_results/pred_500/v0/160_160//../all_v0.tif [Content-Type=image/tiff]...
/ [1/1 files][  1.4 MiB/  1.4 MiB] 100% Done                                    
Operation completed over 1 objects/1.4 MiB.                                      
